# 30 — Error Analysis (μ/σ over 283 bins)

**Goal:** quantify and visualize model errors for Ariel predictions (μ, σ across 283 bins).

**Focus**
- Env detection & path resolution (Kaggle vs local repo)
- Robust loaders for **ground truth** and **predictions**
- Per-bin residual stats, global metrics (MAE/MSE/RMSE, GLL approximation)
- **Uncertainty calibration** checks (coverage, PIT)
- Reliability diagrams (σ vs |error|), residual histograms, per-bin RMSE
- Artifact export: CSV/JSON summaries in `outputs/`

> Pure matplotlib, zero Internet; fast to run.

## 🧭 Environment & Paths

In [ ]:
import os, sys, platform, math, glob, json
from pathlib import Path
from typing import Optional, Dict, Tuple, List
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
BIN_COUNT = 283
COMP_DIR = Path('/kaggle/input/ariel-data-challenge-2025')
REPO_ROOT_CANDIDATES = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
def detect_env() -> Dict:
    env = { 'is_kaggle': COMP_DIR.exists(), 'platform': platform.platform(), 'python': sys.version.replace('\n',' '), 'cwd': str(Path.cwd()), 'repo_root': None }
    for c in REPO_ROOT_CANDIDATES:
        if (c/'configs').exists() and (c/'schemas').exists(): env['repo_root'] = str(c.resolve()); break
    return env
ENV = detect_env()
def resolve_paths(env: Dict) -> Dict[str, Optional[Path]]:
    repo_root = Path(env['repo_root']) if env['repo_root'] else None
    out = { 'competition': COMP_DIR if env['is_kaggle'] else None, 'repo_root': repo_root, 'data_processed': (repo_root/'data'/'processed') if repo_root else None, 'artifacts': (repo_root/'artifacts') if repo_root and (repo_root/'artifacts').exists() else None, 'outputs': Path('outputs') }
    out['outputs'].mkdir(parents=True, exist_ok=True)
    return out
PATHS = resolve_paths(ENV)
ENV, PATHS

## 📥 Load Ground Truth & Predictions
Automatically searches common locations; you can also place CSVs in `outputs/` and name them `*pred*.csv`.

In [ ]:
def find_file(candidates: List[Path]) -> Optional[Path]:
    for p in candidates:
        if p and p.exists(): return p
    return None
gt_candidates: List[Path] = []
pred_candidates: List[Path] = []
if PATHS['competition']: gt_candidates += [PATHS['competition']/ 'train.csv']
if PATHS['data_processed']: gt_candidates += list(PATHS['data_processed'].glob('**/train*.csv'))
if PATHS['artifacts']:
    pred_candidates += list(PATHS['artifacts'].glob('**/*prediction*.csv'))
    pred_candidates += list(PATHS['artifacts'].glob('**/*pred*.csv'))
pred_candidates += list(Path('outputs').glob('*pred*.csv'))
pred_candidates += list(Path('.').glob('*pred*.csv'))
GT_PATH = find_file(gt_candidates)
PRED_PATH = find_file(pred_candidates)
GT_PATH, PRED_PATH

In [ ]:
def load_df(path: Optional[Path], nrows: Optional[int] = None) -> Optional[pd.DataFrame]:
    if not path or not path.exists(): return None
    try:
        if path.suffix == '.csv': return pd.read_csv(path, nrows=nrows)
        elif path.suffix == '.parquet':
            import pyarrow.parquet as pq
            return pq.read_table(path).to_pandas()
    except Exception as e:
        print(f'Failed to load {path}: {e}')
    return None
gt = load_df(GT_PATH)
pred = load_df(PRED_PATH)
print('Ground truth shape:', None if gt is None else gt.shape)
print('Predictions shape:', None if pred is None else pred.shape)

### Column Discovery & Merge
We detect μ/σ columns and a key for alignment; predictions may not include σ.

In [ ]:
def discover_columns(df: pd.DataFrame, kind_prefix: str) -> List[str]:
    return [c for c in df.columns if c.startswith(kind_prefix)]
def find_key(df: pd.DataFrame) -> Optional[str]:
    for k in ('id','sample_id','object_id'):
        if k in df.columns: return k
    return df.columns[0] if len(df.columns) else None
cols = {}
if gt is not None:
    cols['gt_mu'] = discover_columns(gt,'mu_')
    cols['gt_sigma'] = discover_columns(gt,'sigma_')
    cols['key_gt'] = find_key(gt)
if pred is not None:
    cols['pred_mu'] = discover_columns(pred,'mu_')
    cols['pred_sigma'] = discover_columns(pred,'sigma_')
    cols['key_pred'] = find_key(pred)
cols

In [ ]:
issues: List[str] = []
def validate_bin_count(mu_cols: List[str], expected: int = BIN_COUNT) -> None:
    if mu_cols and len(mu_cols) != expected: issues.append(f'Expected {expected} mu_* columns, found {len(mu_cols)}')
if gt is not None: validate_bin_count(cols.get('gt_mu', []))
if pred is not None: validate_bin_count(cols.get('pred_mu', []))
if gt is not None and pred is not None:
    key_gt, key_pred = cols.get('key_gt'), cols.get('key_pred')
    if key_gt is None or key_pred is None:
        issues.append('Could not find join key in GT or PRED.'); merged = None
    else:
        merged = pd.merge(gt, pred, left_on=key_gt, right_on=key_pred, how='inner', suffixes=('_gt','_pred'))
else:
    merged = None
(len(issues), issues), (None if merged is None else merged.shape)

## 🔢 Residuals & Metrics (MAE/MSE/RMSE, approx GLL)

In [ ]:
def residuals_matrix(df: pd.DataFrame, cols_mu_gt: List[str], cols_mu_pred: List[str]) -> Optional[np.ndarray]:
    if df is None or not cols_mu_gt or not cols_mu_pred: return None
    try:
        mu_gt = df[cols_mu_gt].to_numpy(dtype=float); mu_pr = df[cols_mu_pred].to_numpy(dtype=float)
        if mu_gt.shape != mu_pr.shape: return None
        return mu_pr - mu_gt
    except Exception: return None
def global_metrics(res: np.ndarray) -> Dict[str, float]:
    r = res.flatten(); return {'MAE': float(np.mean(np.abs(r))), 'MSE': float(np.mean(r**2)), 'RMSE': float(np.sqrt(np.mean(r**2)))}
def approx_gll(mu_gt: np.ndarray, mu_pr: np.ndarray, sigma: Optional[np.ndarray]) -> float:
    res = mu_pr - mu_gt
    if sigma is None:
        s = res.std(axis=0, ddof=1); sigma = np.maximum(s[None,:], 1e-6)
    else:
        sigma = np.maximum(sigma, 1e-6)
    nll = 0.5*np.log(2*np.pi*sigma**2) + 0.5*(res**2)/(sigma**2)
    return float(np.mean(nll))
res = residuals_matrix(merged, cols.get('gt_mu', []), cols.get('pred_mu', []))
metrics = {}; gll_value = None
if res is not None:
    metrics = global_metrics(res)
    mu_gt = merged[cols['gt_mu']].to_numpy(dtype=float)
    mu_pr = merged[cols['pred_mu']].to_numpy(dtype=float)
    if cols.get('pred_sigma'): sg = merged[cols['pred_sigma']].to_numpy(dtype=float)
    elif cols.get('gt_sigma'): sg = merged[cols['gt_sigma']].to_numpy(dtype=float)
    else: sg = None
    gll_value = approx_gll(mu_gt, mu_pr, sg)
metrics, gll_value

## 📈 Plots (residuals, per-bin RMSE, reliability)

In [ ]:
def plot_hist_residuals(res: Optional[np.ndarray], bins: int = 80):
    if res is None: return
    r = res.flatten(); plt.figure(figsize=(6,4)); plt.hist(r, bins=bins)
    plt.xlabel('residual'); plt.ylabel('count'); plt.title('Residuals (global)'); plt.tight_layout(); plt.show()
def plot_perbin_rmse(res: Optional[np.ndarray]):
    if res is None: return
    rmse = np.sqrt(np.mean(res**2, axis=0)); x = np.arange(len(rmse))
    plt.figure(figsize=(7,4)); plt.plot(x, rmse, lw=1)
    plt.xlabel('bin index'); plt.ylabel('RMSE'); plt.title('Per-bin RMSE'); plt.tight_layout(); plt.show()
def reliability_curve(abs_res: np.ndarray, sigma: np.ndarray, nbins: int = 20):
    order = np.argsort(sigma); abs_res, sigma = abs_res[order], sigma[order]
    edges = np.linspace(sigma.min(), sigma.max(), nbins+1); idx = np.digitize(sigma, edges) - 1
    xs, ys = [], []
    for b in range(nbins):
        m = idx == b
        if m.any(): xs.append(sigma[m].mean()); ys.append(abs_res[m].mean())
    return np.array(xs), np.array(ys)
def plot_reliability(res: Optional[np.ndarray], sigma: Optional[np.ndarray]):
    if res is None or sigma is None: return
    abs_r = np.abs(res).flatten(); s = sigma.flatten()
    if len(s) != len(abs_r): return
    x, y = reliability_curve(abs_r, s, nbins=25)
    plt.figure(figsize=(6,4)); plt.plot(x, y, '.', ms=4)
    plt.xlabel('predicted sigma'); plt.ylabel('mean |residual|'); plt.title('Reliability (|res| vs σ)'); plt.tight_layout(); plt.show()
plot_hist_residuals(res); plot_perbin_rmse(res)
sigma_for_rel = None
if merged is not None and cols.get('pred_sigma'): sigma_for_rel = merged[cols['pred_sigma']].to_numpy(dtype=float)
elif merged is not None and cols.get('gt_sigma'): sigma_for_rel = merged[cols['gt_sigma']].to_numpy(dtype=float)
plot_reliability(res, sigma_for_rel)

## 🎯 Coverage & PIT (Uncertainty Calibration)

In [ ]:
from math import erf
def normal_cdf(z: np.ndarray) -> np.ndarray: return 0.5*(1.0 + erf(z/np.sqrt(2.0)))
def coverage(mu: np.ndarray, sigma: np.ndarray, gt: np.ndarray, k: float) -> float:
    z = np.abs(gt - mu) / np.maximum(sigma, 1e-9); return float(np.mean(z <= k))
coverage_stats = {}; pit_hist = None
if merged is not None and cols.get('pred_mu') and cols.get('gt_mu') and sigma_for_rel is not None:
    mu_pr = merged[cols['pred_mu']].to_numpy(dtype=float); mu_gt = merged[cols['gt_mu']].to_numpy(dtype=float); sg = sigma_for_rel
    coverage_stats['k=1'] = coverage(mu_pr, sg, mu_gt, k=1.0); coverage_stats['k=2'] = coverage(mu_pr, sg, mu_gt, k=2.0)
    z = (mu_gt - mu_pr)/np.maximum(sg, 1e-9); pit = normal_cdf(z).flatten()
    pit_hist = np.histogram(pit, bins=20, range=(0,1))[0].astype(float); pit_hist /= pit_hist.sum()
coverage_stats, (None if pit_hist is None else pit_hist[:5])

In [ ]:
if pit_hist is not None:
    plt.figure(figsize=(6,4))
    plt.bar(np.linspace(0.025,0.975,20), pit_hist, width=0.045)
    plt.xlabel('PIT bin center'); plt.ylabel('frequency'); plt.title('PIT histogram (should be ~uniform)')
    plt.tight_layout(); plt.show()

## 🧾 Export Reports

In [ ]:
summary = {
    'env': ENV,
    'shapes': {
        'gt': None if gt is None else list(gt.shape),
        'pred': None if pred is None else list(pred.shape),
        'merged': None if merged is None else list(merged.shape),
    },
    'metrics': metrics,
    'gll': gll_value,
    'coverage': coverage_stats,
    'issues': issues,
}
Path('outputs').mkdir(exist_ok=True, parents=True)
with open('outputs/error_summary.json', 'w', encoding='utf-8') as f: json.dump(summary, f, indent=2)
if res is not None:
    rmse = np.sqrt(np.mean(res**2, axis=0))
    pd.DataFrame({'bin': np.arange(len(rmse)), 'rmse': rmse}).to_csv('outputs/per_bin_rmse.csv', index=False)
print('Wrote outputs/error_summary.json and (optionally) outputs/per_bin_rmse.csv')

## Next Steps
- Harmonize column names with your submission format and training pipeline
- Add CRPS or weighted GLL consistent with the challenge metric if needed
- Integrate with DVC stage to track metrics/artifacts across runs

**Done.** ✅